##MLflow Quick Start Part 2: Serving Models with Microsoft Azure ML

This notebook is part of a Quick Start guide based on the [MLflow tutorial](https://www.mlflow.org/docs/latest/tutorial.html).
The [first part of the guide](https://docs.azuredatabricks.net/spark/latest/mllib/mlflow.html), **MLflow Quick Start: Model Training and Logging**, focuses on training a model and logging the training metrics, parameters, and model to the MLflow tracking server. 

##### NOTE: We do not recommend using *Run All* because it takes several minutes to deploy and update models; models cannot be queried until they are active.

This part of the guide consists of the following sections:

#### Setup
* Launch an Azure Databricks cluster
* Install MLflow
* Install the Azure ML SDK
* Create or load an Azure ML Workspace

#### Building an Azure Container Image for model deployment
* Use MLflow to build a Container Image for the trained model

#### Deploying the model to "dev" using Azure Container Instances (ACI)
* Create an ACI webservice deployment using the model's Container Image

#### Querying the deployed model in "dev"
* Load a sample input vector from the diabetes dataset
* Evaluate the sample input vector by sending an HTTP request

#### Deploying the model to production using Azure Kubernetes Service (AKS)
* Option 1: Create a new AKS cluster
* Option 2: Connect to an existing AKS cluster
* Deploy to the model's image to the specified AKS cluster

#### Querying the deployed model in production
* Load a sample input vector from the wine dataset
* Evaluate the sample input vector by sending an HTTP request

#### Updating the production deployment
* Build an Azure Container Image for another model
* Deploy the new model's image to the AKS cluster
* Query the updated model

#### Cleaning up the deployments
* Terminate the "dev" ACI webservice
* Terminate the production AKS webservice
* Remove the AKS cluster from the Azure ML Workspace

As in the first part of the Quick Start tutorial, this notebook uses ElasticNet models trained on the `diabetes` dataset in scikit-learn.

**Note:** This notebook expects that you use an Azure Databricks hosted MLflow tracking server. If you would like to preview the Azure Databricks MLflow tracking server, contact your Azure Databricks sales representative to request access. To set up your own tracking server, see the instructions in [MLflow Tracking Servers](https://www.mlflow.org/docs/latest/tracking.html#mlflow-tracking-servers) and configure your connection to your tracking server by running [mlflow.set_tracking_uri](https://www.mlflow.org/docs/latest/python_api/mlflow.html#mlflow.set_tracking_uri).

## Prerequisites
ElasticNet models from the MLflow Quick Start notebook in [part 1 of the Quick Start guide](https://docs.azuredatabricks.net/applications/mlflow/mlflow-training.html).

## Setup

1. Ensure you are using or create a cluster specifying 
  * **Databricks Runtime Version:** Databricks Runtime 5.0 or above
  * **Python Version:** Python 3
1. Install required libraries or if using Databricks Runtime 5.1 or above (but not Databricks Runtime for ML), run Cmd 6.
   1. Create required libraries.
      * Source **PyPI** and enter `mlflow`.
      * Source **PyPI** and enter `azureml-sdk[databricks]`.
      * Source **PyPI** and enter `scikit-learn==0.19.1`.
   1. Install the libraries into the cluster.
1. Attach this notebook to the cluster.

In [6]:
dbutils.library.installPyPI("mlflow")
dbutils.library.installPyPI("scikit-learn", "0.19.1")
dbutils.library.installPyPI("azureml-sdk")
dbutils.library.restartPython()

### Create or load an Azure ML Workspace

Before models can be deployed to Azure ML, you must create or obtain an Azure ML Workspace. The `azureml.core.Workspace.create()` function will load a workspace of a specified name or create one if it does not already exist. For more information about creating an Azure ML Workspace, see the [Azure ML Workspace management documentation](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-manage-workspace).

In [9]:
import azureml
from azureml.core import Workspace

workspace_name = "DataBricks_Model_Deployment"
workspace_location="eastus"
resource_group = "DataBricks_RG"
subscription_id = "49821f6d-b40c-4ddc-8dff-dd75b3729732"

workspace = Workspace.create(name = workspace_name,
                             location = workspace_location,
                             resource_group = resource_group,
                             subscription_id = subscription_id,
                             exist_ok=True)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code AQK27RGHF to authenticate.
Interactive authentication successfully completed.
Deploying AppInsights with name databricinsightsutuvujet.
Deployed AppInsights with name databricinsightsutuvujet.
Deploying KeyVault with name databrickeyvaultururrewj.
Deploying StorageAccount with name databricstoragexsgxlyrd.
Deploying ContainerRegistry with name databricacrofubcqoq.
Deployed ContainerRegistry with name databricacrofubcqoq.
Deployed KeyVault with name databrickeyvaultururrewj.
Deployed StorageAccount with name databricstoragexsgxlyrd.
Deploying Workspace with name DataBricks_Model_Deployment.
Deployed Workspace with name DataBricks_Model_Deployment.

## Building an Azure Container Image for model deployment

### Use MLflow to build a Container Image for the trained model

Use the `mlflow.azuereml.build_image` function to build an Azure Container Image for the trained MLflow model. This function also registers the MLflow model with a specified Azure ML workspace. The resulting image can be deployed to Azure Container Instances (ACI) or Azure Kubernetes Service (AKS) for real-time serving.

## Review the experiment

1. Open the experiment `/Shared/experiments/DiabetesModel` in the workspace.
1. Click a date to view a run.

Specify the run ID associated with an ElasticNet training run from [part 1 of the Quick Start guide](https://docs.azuredatabricks.net/spark/latest/mllib/mlflow-tracking.html). You can find a run ID and model path from the experiment run, which can be found on the run details page:

![image](https://docs.azuredatabricks.net/_static/images/mlflow/mlflow-deployment-example-run-info.png)

In [14]:
run_id1="61f3a15564cd436f9ac8d110834a9e8d"
model_path = "model"

In [15]:
import mlflow.azureml

model_image, azure_model = mlflow.azureml.build_image(model_path=model_path, 
                                                      workspace=workspace, 
                                                      run_id=run_id1,
                                                      model_name="model",
                                                      image_name="model",
                                                      description="Sklearn ElasticNet image for predicting diabetes progression",
                                                      synchronous=False)
#image_config is default(reserach on getting the environment)

/databricks/python/lib/python3.5/site-packages/mlflow/utils/environment.py:26: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
 env = yaml.load(_conda_header)
Registering model model
Creating image

In [16]:
model_image.wait_for_creation(show_output=True)

Running................................................................
SucceededImage creation operation finished for image model:1, operation "Succeeded"

## Deploying the model to "dev" using [Azure Container Instances (ACI)](https://docs.microsoft.com/en-us/azure/container-instances/)

The [ACI platform](https://docs.microsoft.com/en-us/azure/container-instances/) is the recommended environment for staging and developmental model deployments.

### Create an ACI webservice deployment using the model's Container Image

Using the Azure ML SDK, deploy the Container Image for the trained MLflow model to ACI.

In [19]:
from azureml.core.webservice import AciWebservice, Webservice

dev_webservice_name = "diabetes-model"
dev_webservice_deployment_config = AciWebservice.deploy_configuration()
dev_webservice = Webservice.deploy_from_image(name=dev_webservice_name, image=model_image, deployment_config=dev_webservice_deployment_config, workspace=workspace)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-552820134171463> in <module> () 
 3 dev_webservice_name = "diabetes-model" 
 4 dev_webservice_deployment_config = AciWebservice . deploy_configuration ( ) 
 ----> 5 dev_webservice = Webservice . deploy_from_image ( name = dev_webservice_name , image = model_image , deployment_config = dev_webservice_deployment_config , workspace = workspace ) 

 NameError : name 'model_image' is not defined

In [20]:
dev_webservice_deployment_config

Out[ 27 ]: <azureml.core.webservice.aci.AciServiceDeploymentConfiguration at 0x7f7e6481a4e0>

In [21]:
dev_webservice.wait_for_deployment()

ACI service creation operation finished, operation "Succeeded"

## Querying the deployed model in "dev"

### Load diabetes dataset

In [24]:
from sklearn import datasets
diabetes = datasets.load_diabetes()

## Create sample input vector

In [26]:
import pandas as pd
import numpy as np

X = diabetes.data
y = diabetes.target
Y = np.array([y]).transpose()
d = np.concatenate((X, Y), axis=1)
cols = ['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6', 'progression']
data = pd.DataFrame(d, columns=cols)
sample = data.drop(["progression"], axis=1).iloc[[0]]
                                                 
query_input = sample.to_json(orient='split')
query_input = eval(query_input)
query_input.pop('index', None)

Out[ 7 ]: [0]

In [27]:
query_input

Out[ 8 ]: 
{'columns': ['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6'],
 'data': [[0.0380759064,
 0.0506801187,
 0.0616962065,
 0.021872355,
 -0.0442234984,
 -0.0348207628,
 -0.0434008457,
 -0.002592262,
 0.0199084209,
 -0.0176461252]]}

#### Evaluate the sample input vector by sending an HTTP request
Query the ACI webservice's scoring endpoint by sending an HTTP POST request that contains the input vector.

In [29]:
import requests
import json

def query_endpoint_example(scoring_uri, inputs, service_key=None):
  headers = {
    "Content-Type": "application/json",
  }
  if service_key is not None:
    headers["Authorization"] = "Bearer {service_key}".format(service_key=service_key)
    
  print("Sending batch prediction request with inputs: {}".format(inputs))
  response = requests.post(scoring_uri, data=json.dumps(inputs), headers=headers)
  preds = json.loads(response.text)
  print("Received response: {}".format(preds))
  return preds

In [30]:
dev_webservice.scoring_uri

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-552820134171472> in <module> () 
 ----> 1 dev_webservice . scoring_uri 

 NameError : name 'dev_webservice' is not defined

In [31]:
dev_prediction = query_endpoint_example(scoring_uri='http://52.151.228.88:80/score', inputs=query_input)
#dev_prediction = query_endpoint_example(scoring_uri=dev_webservice.scoring_uri, inputs=query_input)

Sending batch prediction request with inputs: {'columns': ['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6'], 'data': [[0.0380759064, 0.0506801187, 0.0616962065, 0.021872355, -0.0442234984, -0.0348207628, -0.0434008457, -0.002592262, 0.0199084209, -0.0176461252]]}
Received response: [209.994170046977]

## Deploying the model to production using [Azure Kubernetes Service (AKS)](https://azure.microsoft.com/en-us/services/kubernetes-service/). Do Option 1 or Option 2.

### Option 1: Create a new AKS cluster

If you do not have an active AKS cluster for model deployment, create one using the Azure ML SDK.

In [34]:
from azureml.core.compute import AksCompute, ComputeTarget

# Use the default configuration (you can also provide parameters to customize this)
prov_config = AksCompute.provisioning_configuration()

aks_cluster_name = "diabetes-cluster" 
# Create the cluster
aks_target = ComputeTarget.create(workspace = workspace, 
                                  name = aks_cluster_name, 
                                  provisioning_configuration = prov_config)

# Wait for the create process to complete
aks_target.wait_for_completion(show_output = True)
print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)

Creating........................................................................................................................................................................
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
None

### Option 2: Connect to an existing AKS cluster

If you already have an active AKS cluster running, you can add it to your Workspace using the Azure ML SDK.

In [36]:
from azureml.core.compute import AksCompute, ComputeTarget

# Get the resource id from https://porta..azure.com -> Find your resource group -> click on the Kubernetes service -> Properties
resource_id = "/subscriptions/<subscription-id>/resourcegroups/<resource-group>/providers/Microsoft.ContainerService/managedClusters/<aks-service-name>"

# Give the cluster a local name
cluster_name = "<cluster-name>"

# Attatch the cluster to your workgroup
aks_target = AksCompute.attach(workspace=workspace, name=cluster_name, resource_id=resource_id)

# Wait for the operation to complete
aks_target.wait_for_completion(True)
print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)

### Deploy to the model's image to the specified AKS cluster

In [38]:
from azureml.core.webservice import Webservice, AksWebservice

# Set configuration and service name
prod_webservice_name = "diabetes-model-prod"
#prod_webservice_name = "SGIprod"
prod_webservice_deployment_config = AksWebservice.deploy_configuration()

# Deploy from image
prod_webservice = Webservice.deploy_from_image(workspace = workspace, 
                                               name = prod_webservice_name,
                                               image = model_image,
                                               deployment_config = prod_webservice_deployment_config,
                                               deployment_target = aks_target)

Creating service

In [39]:
# Wait for the deployment to complete
prod_webservice.wait_for_deployment(show_output = True)

Running.................
SucceededAKS service creation operation finished, operation "Succeeded"

## Querying the deployed model in production

#### Evaluate the sample input vector by sending an HTTP request
Query the AKS webservice's scoring endpoint by sending an HTTP POST request that includes the input vector. The production AKS deployment may require an authorization token (service key) for queries. Include this key in the HTTP request header.

In [42]:
import requests
import json

def query_endpoint_example(scoring_uri, inputs, service_key=None):
  headers = {
    "Content-Type": "application/json",
  }
  if service_key is not None:
    headers["Authorization"] = "Bearer {service_key}".format(service_key=service_key)
    
  print("Sending batch prediction request with inputs: {}".format(inputs))
  response = requests.post(scoring_uri, data=json.dumps(inputs), headers=headers)
  preds = json.loads(response.text)
  print("Received response: {}".format(preds))
  return preds

In [43]:
prod_scoring_uri = prod_webservice.scoring_uri
prod_service_key = prod_webservice.get_keys()[0] if len(prod_webservice.get_keys()) > 0 else None

In [44]:
prod_prediction1 = query_endpoint_example(scoring_uri=prod_scoring_uri, service_key=prod_service_key, inputs=query_input)

Sending batch prediction request with inputs: {'data': [[0.0380759064, 0.0506801187, 0.0616962065, 0.021872355, -0.0442234984, -0.0348207628, -0.0434008457, -0.002592262, 0.0199084209, -0.0176461252]], 'columns': ['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']}
Received response: [209.994170046977]

## Updating the production deployment

### Build an Azure Container Image for the new model

In [47]:
run_id2="2d622de93c334322a63ed1c9b7ee0846"

In [48]:
import mlflow.azureml

model_image_updated, azure_model_updated = mlflow.azureml.build_image(model_path="model", 
                                                                      workspace=workspace, 
                                                                      run_id=run_id2,
                                                                      model_name="model-updated",
                                                                      image_name="model-updated",
                                                                      description="Sklearn ElasticNet image for predicting diabetes progression",
                                                                      synchronous=False)

Registering model model-updated
Creating image

In [49]:
model_image_updated.wait_for_creation(show_output=True)

Running........................................................
SucceededImage creation operation finished for image model-updated:1, operation "Succeeded"

### Deploy the new model's image to the AKS cluster

Using the [`azureml.core.webservice.AksWebservice.update()`](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.webservice.akswebservice?view=azure-ml-py#update) function, replace the deployment's existing model image with the new model image.

In [51]:
prod_webservice.update(image=model_image_updated)

In [52]:
prod_webservice.wait_for_deployment(show_output = True)

Running............
SucceededAKS service creation operation finished, operation "Succeeded"

### Query the updated model

In [54]:
prod_prediction2 = query_endpoint_example(scoring_uri=prod_scoring_uri, service_key=prod_service_key, inputs=query_input)

Sending batch prediction request with inputs: {'data': [[0.0380759064, 0.0506801187, 0.0616962065, 0.021872355, -0.0442234984, -0.0348207628, -0.0434008457, -0.002592262, 0.0199084209, -0.0176461252]], 'columns': ['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']}
Received response: [181.73269679516147]

## Compare the predictions

In [56]:
print("Run ID: {} Prediction: {}".format(run_id1, prod_prediction1)) 
print("Run ID: {} Prediction: {}".format(run_id2, prod_prediction2))

Run ID: 61f3a15564cd436f9ac8d110834a9e8d Prediction: [209.994170046977]
Run ID: 2d622de93c334322a63ed1c9b7ee0846 Prediction: [181.73269679516147]

## Cleaning up the deployments

### Terminate the "dev" ACI webservice

Because ACI manages compute resources on your behalf, deleting the "dev" ACI webservice will remove all resources associated with the "dev" model deployment

In [59]:
dev_webservice.delete()

### Terminate the production AKS webservice

This terminates the real-time serving webservice running on the specified AKS cluster. It **does not** terminate the AKS cluster.

In [61]:
prod_webservice.delete()

### Remove the AKS cluster from the Azure ML Workspace

If the cluster was created using the Azure ML SDK (see **Option 1: Create a new AKS cluster**), remove it from the Azure ML Workspace will terminate the cluster, including all of its compute resources and deployments.

If the cluster was created independently (see **Option 2: Connect to an existing AKS cluster**), it will remain active after removal from the Azure ML Workspace.

In [63]:
aks_target.delete()